# 🧠 Урок 28: Оптимизация и регуляризация моделей
**Цель урока:** Понять, как улучшить качество модели с помощью оптимизации гиперпараметров, регуляризации (Dropout, L2) и нормализации (BatchNorm). Подходит для новичков.

## 📌 Почему модели переобучаются?
- **Переобучение (Overfitting):** Модель запоминает тренировочные данные, но плохо обобщает на тестовых.
- **Причины:** Слишком много нейронов, мало данных, шум в данных.
- **Аналогия:** Представьте, что вы зубрили ответы на вопросы, а на экзамене они изменились — вы провалите тест.

## 📉 Что такое недообучение (Underfitting)?
- **Недообучение:** Модель не выучила даже основные закономерности.
- **Причины:** Слишком простая модель, недостаток данных.
- **Аналогия:** Если вы читаете учебник один раз, но не поняли тему — это недообучение.

## 🧰 Методы оптимизации
### 1. Градиентный спуск
- **Что это?** Алгоритм, который минимизирует функцию потерь, двигаясь в направлении градиента.
- **Формула обновления весов:**
  ```python
  weights -= learning_rate * gradient
  ```
- **learning_rate:** Шаг обучения. Слишком большой — модель не сходится, слишком маленький — медленно обучается.
- **Аналогия:** Вы спускаетесь с горы, делая маленькие или большие шаги. Оптимальный шаг — это золотая середина.

### 2. Adam (Adaptive Moment Estimation)
- **Что это?** Оптимизатор, который адаптирует learning rate для каждого параметра.
- **Преимущества:** Быстро сходится, работает на больших данных.
- **Пример:**
  ```python
  from tensorflow.keras.optimizers import Adam
  optimizer = Adam(learning_rate=0.001)
  ```
- **Аналогия:** Adam — как GPS, который подстраивает маршрут в зависимости от пробок.

## 🛡️ Регуляризация для борьбы с переобучением
### 1. L2-регуляризация (Ridge)
- **Что это?** Добавление штрафа за большие веса:
  ```python
  model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
  ```
- **Зачем?** Чтобы модель не запоминала данные, а находила общие закономерности.
- **Аналогия:** L2-регуляризация — как ограничение на расходы: вы не можете тратить слишком много на отдельные параметры.

### 2. Dropout
- **Что это?** Случайное отключение нейронов во время обучения.
- **Зачем?** Чтобы модель не зависела от конкретных нейронов.
- **Пример:**
  ```python
  from tensorflow.keras.layers import Dropout
  model.add(Dropout(0.5))  # Отключает 50% нейронов
  ```
- **Аналогия:** Dropout — как тренировка команды без одного игрока: остальные учатся компенсировать его отсутствие.

### 3. Batch Normalization
- **Что это?** Нормализация входных данных на каждом слое.
- **Зачем?** Ускоряет обучение и снижает чувствительность к инициализации весов.
- **Пример:**
  ```python
  from tensorflow.keras.layers import BatchNormalization
  model.add(BatchNormalization())
  ```
- **Аналогия:** BatchNorm — как коррекция зрения: вы всегда видите четко, независимо от освещения.

### 4. Early Stopping
- **Что это?** Остановка обучения, если качество на валидации не улучшается.
- **Пример:**
  ```python
  from tensorflow.keras.callbacks import EarlyStopping
  early_stop = EarlyStopping(monitor='val_loss', patience=3)
  model.fit(X_train, y_train, callbacks=[early_stop])
  ```
- **Аналогия:** Early Stopping — как перестать учить, если вы уже знаете тему, а новые данные не помогают.

## 🧪 Практика: Обучение с регуляризацией
### Шаг 1: Загрузка и подготовка данных

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

# Загрузка MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']
X = X / 255.0  # Нормализация
y = np.eye(10)[y.astype(int)]  # One-hot кодирование

# Разделение
from sklearn.model_selection import train_test_split
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

### Шаг 2: Создание модели с BatchNorm и Dropout

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

### Шаг 3: Компиляция и обучение

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, 
                  epochs=20,
                  batch_size=128,
                  validation_data=(X_test, y_test))

### Шаг 4: Визуализация обучения

In [ ]:
import matplotlib.pyplot as plt

# График точности
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.legend()
plt.title('Accuracy по эпохам')
plt.xlabel('Эпохи')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## 📈 Как работает BatchNorm?
- **BatchNorm:** Нормализует выходы слоев, чтобы среднее было 0, дисперсия — 1.
- **Зачем?** Ускоряет обучение, снижает зависимость от инициализации весов.
- **Пример:**
  ```python
  from tensorflow.keras.layers import BatchNormalization
  model.add(BatchNormalization())
  ```
- **Аналогия:** BatchNorm — как автоматическая настройка рояля: вы играете, и инструмент сам подстраивается под вас.

## 📊 Как работает Dropout?
- **Dropout:** Во время обучения случайно отключает нейроны, чтобы предотвратить переобучение.
- **Пример:**
  ```python
  from tensorflow.keras.layers import Dropout
  model.add(Dropout(0.5))  # Отключает 50% нейронов
  ```
- **Аналогия:** Dropout — как тренировка команды без одного игрока: остальные учатся играть лучше.

## 📈 Как работает Adam?
- **Adam:** Адаптивный оптимизатор, который меняет learning rate для каждого параметра.
- **Преимущества:** Быстрое обучение, работает на больших данных.
- **Параметры:**
  - `learning_rate`: Скорость обучения.
  - `beta_1`, `beta_2`: Параметры для вычисления моментов.
- **Пример:**
  ```python
  from tensorflow.keras.optimizers import Adam
  optimizer = Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  ```
- **Аналогия:** Adam — как инструктор, который подстраивает темп под ваши способности.

## 📊 Как выбрать лучшие гиперпараметры?
- **GridSearchCV:** Перебор всех возможных комбинаций.
- **RandomizedSearchCV:** Случайный перебор (быстрее, чем GridSearch).
- **Пример:**
  ```python
  from sklearn.model_selection import GridSearchCV
  param_grid = {'learning_rate': [0.001, 0.01], 'batch_size': [32, 64]}
  grid = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')
  grid.fit(X_train, y_train)
  ```
- **Аналогия:** Подбор гиперпараметров — как поиск идеального рецепта через эксперименты с ингредиентами.

## 📝 Домашнее задание
**Задача 1:** Попробуйте разные значения Dropout: 0.2, 0.5, 0.7. Как меняется точность?
**Задача 2:** Добавьте L2-регуляризацию в первый слой и сравните результаты.
**Задача 3:** Напишите отчет (200–300 слов), где:
- Опишите, как вы использовали BatchNorm и Dropout.
- Сравните точность модели до и после регуляризации.
- Объясните, почему эти методы помогают.
- Приведите примеры, где они полезны (например, в больших нейросетях, на малых данных).

In [ ]:
# Ваш код здесь
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Модель с разными Dropout
def build_model(dropout_rate=0.5):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(784,)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Обучение с разными Dropout
model_0_2 = build_model(0.2)
history_0_2 = model_0_2.fit(X_train, y_train, 
                           epochs=10,
                           batch_size=64,
                           validation_data=(X_test, y_test))

In [ ]:
# Обучение с L2-регуляризацией
from tensorflow.keras.regularizers import l2

model_l2 = Sequential([
    Dense(128, activation='relu', input_shape=(784,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model_l2.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
history_l2 = model_l2.fit(X_train, y_train, 
                         epochs=10,
                         batch_size=64,
                         validation_data=(X_test, y_test))

In [ ]:
# Сравнение точности
import matplotlib.pyplot as plt

plt.plot(history_0_2.history['val_accuracy'], label='Dropout 0.2')
plt.plot(history_l2.history['val_accuracy'], label='L2 Regularization')
plt.legend()
plt.title('Сравнение регуляризации')
plt.xlabel('Эпохи')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## ✅ Рекомендации по выполнению
- **Задача 1:** После изменения Dropout следите за `val_loss` и `val_accuracy`.
- **Задача 2:** Добавьте L2-регуляризацию в разные слои и сравните результаты.
- **Подсказка:** Используйте `model.summary()` для анализа архитектуры.